# Homework 4 - GIS5090
## Carter Hanford

Homework 4 is a walkthrough of exercise 07 from `Zandbergen`, which serves as an introduction to using cursors within ArcPro.

### Part 1 - printvalues.py
This section will create a python script which reads and prints all values from the airport shapefile.

In [4]:
%%time

import arcpy
from arcpy import env
arcpy.env.workspace = r'C:\Users\hanfordca\Documents\GitHub\python-programming-spring-2020\week 4 - feature data in arcpy\data\Exercise07'

# creating a feature class object for airports
fc = "airports.shp"
cursor = arcpy.da.SearchCursor(fc, ["NAME"])

# for loop which prints all airport names
for row in cursor:
    print ("Airport name = {0}".format(row[0]))

Airport name = Hyder
Airport name = Chignik Lagoon
Airport name = Koyuk
Airport name = Kivalina
Airport name = Ketchikan Harbor
Airport name = Metlakatla
Airport name = Waterfall
Airport name = Ketchikan
Airport name = Kasaan
Airport name = Hollis
Airport name = Craig
Airport name = Thorne Bay
Airport name = Coffman Cove
Airport name = Meyers Chuck
Airport name = Klawock
Airport name = Yes Bay Lodge
Airport name = Wrangell
Airport name = Petersburg James A. Johnson
Airport name = Kake
Airport name = Sitka Rocky Gutierrez
Airport name = Angoon
Airport name = Funter Bay
Airport name = Tenakee
Airport name = Hawk Inlet
Airport name = Hoonah
Airport name = Juneau International
Airport name = Pelican
Airport name = Elfin Cove
Airport name = Excursion Inlet
Airport name = Gustavus
Airport name = Haines
Airport name = Skagway
Airport name = Yakutat
Airport name = Icy Bay
Airport name = Alitak
Airport name = Akhiok
Airport name = San Juan/Uganik Bay
Airport name = Old Harbor
Airport name = Lar

### Part 2 - SQL.py
This section creates a python script where only the names of the airports for which the SQL expression is true are printed.

In [5]:
# we do not need to import arcpy or set the workspace again
cursor = arcpy.da.SearchCursor(fc, ["NAME"], '"TOT_ENP" > 100000')

for row in cursor:
    print (row[0])

del row
del cursor

Ketchikan
Juneau International
Kenai Municipal
Fairbanks International
Bethel
Ted Stevens Anchorage International


The syntax in the code above can cause issues, according to the exercise, so we'll need to modify the script so that python knows when we are working with numeric fields or text fields, and thus not produce a syntax error. We'll use `\` to work around this problem.

In [7]:
cursor = arcpy.da.SearchCursor(fc, ["NAME"], '"FEATURE" = \'Seaplane Base\'')

for row in cursor:
    print (row[0])

Hyder
Ketchikan Harbor
Metlakatla
Waterfall
Kasaan
Hollis
Craig
Thorne Bay
Coffman Cove
Meyers Chuck
Yes Bay Lodge
Kake
Angoon
Funter Bay
Tenakee
Hawk Inlet
Pelican
Elfin Cove
Excursion Inlet
Alitak
San Juan/Uganik Bay
Trident Basin
Ivanof Bay
Lake Brooks
Lake Hood
Akutan
Akiachak
Newtok


There are other syntax issues when working with SQL expressions, such as working with shapefiles and file geodatabases, so we'll ammend our script again to work with field delimiters. The ammended script below corresponds with the `select.py` section of the exercise.

In [9]:
# no need to import arcpy or our workspace environment
delimitedField = arcpy.AddFieldDelimiters(fc, "COUNTY")


# set cursor
cursor = arcpy.da.SearchCursor(fc, ["NAME"], delimitedField + 
                               " = 'Anchorage Borough'")

# create script
for row in cursor:
    print(row[0])

del row
del cursor

Girdwood
Merrill Field
Lake Hood
Elmendorf Air Force Base
Ted Stevens Anchorage International


We will now save the new fields into our `results` folder:

In [10]:
# create input and output feature class
infc = "airports.shp"
outfc = "Results/airports_anchorage.shp"

delimitedField = arcpy.AddFieldDelimiters(infc, "COUNTY")
arcpy.Select_analysis(infc, outfc, delimitedField + 
                      " = 'Anchorage Borough'")

<Result 'C:\\Users\\hanfordca\\Documents\\GitHub\\python-programming-spring-2020\\week 4 - working with feature data in arcpy\\data\\Exercise07\\Results\\airports_anchorage.shp'>

By checking the results folder in our workspace directory, I can confirm that the new anchorage borough shapefiles are indeed in the correct folder!

### Part 3 - update.py

In [12]:
# we don't need to import arcpy or our workspace env
fc = "Results/airports.shp"
delimField = arcpy.AddFieldDelimiters(fc, "STATE")

# set cursor
cursor = arcpy.da.UpdateCursor(fc, ["STATE"], delimField + 
                              " <> 'AK'")

# for loop
for row in cursor:
    row[0] = "AK"
    cursor.updateRow(row)

del row
del cursor

This python script takes the `STATE` column in our airport shapefile and updates the fields to include Alaska.

### Part 4 - delete.py

In [3]:
# we can use the same feature class object
fc = "Results/airports.shp"
cursor = arcpy.da.UpdateCursor(fc, ["TOT_ENP"])

# for loop
for row in cursor:
    if row[0] < 100000:
        cursor.deleteRow()
        
del row
del cursor

### Part 5 - insert.py

In [4]:
# we can use the same feature class object
arcpy.env.workspace = r'C:\Users\hanfordca\Documents\GitHub\python-programming-spring-2020\week 4 - feature data in arcpy\data\Exercise07'

fc = "Results/airports.shp"
cursor = arcpy.da.InsertCursor(fc, "NAME")
cursor.insertRow(["New Airport"])
del cursor

Part 4 and 5 of the exercise use `update` and `insert` cursors to change and work with values in our shapefiles. 

### Part 6 - validatefield.py

In [5]:
fc = "Results/airports.shp"
newfield = "NEW CODE"
fieldtype = "TEXT"
fieldname = arcpy.ValidateFieldName(newfield)

arcpy.AddField_management(fc, fieldname, fieldtype, "", "", 12)

<Result 'C:\\Users\\hanfordca\\Documents\\GitHub\\python-programming-spring-2020\\week 4 - working with feature data in arcpy\\data\\Exercise07\\Results/airports.shp'>

We can confirm this worked by opening ArcPro and checking the attribute table for the airport shapefile. We can modify the code slightly to replace all invalid characters with an underscore:

In [6]:
fc = "Results/airports.shp"
newfield = "NEW?*&$"
fieldtype = "TEXT"
fieldname = arcpy.ValidateFieldName(newfield)

arcpy.AddField_management(fc, fieldname, fieldtype, "", "", 12)

<Result 'C:\\Users\\hanfordca\\Documents\\GitHub\\python-programming-spring-2020\\week 4 - working with feature data in arcpy\\data\\Exercise07\\Results/airports.shp'>

We can modify this even further:

In [8]:
newfield = "NEW CODE"
fieldtype = "TEXT"
fieldname = arcpy.ValidateFieldName(newfield)
fieldlist = arcpy.ListFields(fc)
fieldnames = []

# for loop
for field in fieldlist:
    fieldnames.append(field.name)
if fieldname not in fieldnames:
    arcpy.AddField_management(fc, fieldname, fieldtype, "", "", 12)
    print ("New field has been added.")
else: 
    print ("Field name already exists.")

Field name already exists.


### Part 7 - unique_name.py

In [11]:
fc = "airports.shp"
unique_name = arcpy.CreateUniqueName("Results/buffer.shp")

# create buffer shapefile
arcpy.Buffer_analysis(fc, unique_name, "5000 METERS")

<Result 'C:\\Users\\hanfordca\\Documents\\GitHub\\python-programming-spring-2020\\week 4 - working with feature data in arcpy\\data\\Exercise07\\Results\\buffer0.shp'>

We can confirm this code worked by checking our results folder for the new shapefile. Each time we run this code, it creates a new buffer shapefile which looks like this:
 * buffer0.shp
 * buffer1.shp
 * buffer(n).shp for n times

### Challenge 2
Challenge two asks us to write a script that adds a text field to the roads shapefile called `FERRY` and populates the field with YES and NO depending on the value of the `FEATURE` field.

In [21]:
fc = "roads.shp"
newfield = "FERRY"
fieldtype = "TEXT"
fieldname = arcpy.ValidateFieldName(newfield)
fieldlist = arcpy.ListFields(fc)
arcpy.AddField_management(fc, fieldname, fieldtype, "", "", 12)

cursor = arcpy.da.UpdateCursor(fc, ["FEATURE", "FERRY"])

# create for loop to populate field in FERRY
for row in cursor:
    if row[0] == "Ferry Crossing":
        row[1] = "YES"
    else:
        row[1] = "NO"
    cursor.updateRow(row)
    
del row
del cursor